# Emerging Technolgoies

### Libraries 

In [7]:
# Random selections.
import random

# Numerical arrays.
import numpy as np

## Problem 1: Generating Random Boolean Functions


### Define the four constant and balanced functions 

In [8]:
# Constant function 1.
# Constant functions do not depend on any input
# they always return the same value (0 or 1), regardless of
# the values of their arguments.
def f1(a,b,c,d):
  """Takes a single bit but always returns 0."""
  return 0

# Balanced function 1.
def f2(a,b,c,d):
  """Identity function."""
  return a

# Balanced function 2.
def f3(a,b,c,d):
  """Not function."""
  return a ^ b ^ c ^ d

# constant function 2. 
def f4(a,b,c,d):
  """Constant 1 function."""
  return 1

### The random_constant_balanced function returns a randomly selected function 

In [9]:
def random_constant_balanced():
  """Randomly returns one of the four functions defined above."""
  # List of potential functions.
  functions = [f1, f2, f3, f4]
  # Return element at random.
  random_function = random.choice(functions)
  return random_function


## Problem 2: Classical Testing for Function Type


In [ ]:
def determine_constant_balanced(random_function):
    results = []
    # all 16 possible inputs for the function 
    inputs = [(a, b, c, d) for a in [0, 1] for b in [0, 1] for c in [0, 1] for d in [0, 1]]

    for input in inputs:
        # for each input, call the random function and store the output in results
        output = random_function(*input)
        results.append(output)

        ones = results.count(1)
        zeros = results.count(0)

        # balanced if there are equal number of 1s and 0s
        if ones > 0 and zeros > 0:
            return ("Balanced: ", results)

        # with only 16 possible inputs if there are more than 8 1s or 0s then the function is constant
        if ones > 8 or zeros > 8:
            return ("Constant: ", results)


In [62]:
# Get a random function
function = random_constant_balanced()
# Determine if the function is constant or balanced
test = determine_constant_balanced(function)
test


('Balanced: ', [0, 0, 0, 0, 0, 0, 0, 0, 1])

### What is the maximum number of times you need to call f to be 100% certain whether it is balanced?

A function accepting 4 arguments as inputs can only have 16 possible inputs - 8 constant inputs and 8 balanced inputs 

- **The 16 possible inputs**

    (0, 0, 0, 0), (0, 0, 0, 1), (0, 0, 1, 0), (0, 0, 1, 1), (0, 1, 0, 0), (0, 1, 0, 1), (0, 1, 1, 0), (0, 1, 1, 1),
    (1, 0, 0, 0), (1, 0, 0, 1), (1, 0, 1, 0), (1, 0, 1, 1), (1, 1, 0, 0), (1, 1, 0, 1), (1, 1, 1, 0), (1, 1, 1, 1)

`Balanced Functions` : A function that returns 0 exactly half of the time and exactly 1 have of the time.

**Minimum**

With a function that accepts 4 argumnets as input the minimum number of times it would need to call the function to determine if it is balanced is 2. If the first time the random function returns 0 and the second time it returns 1, this is enough to classify the function as balanced as a constant function will only ever return 1 or only ever return 0.

**Maximum**

The maximum number of times you would need to call f to determine if it is balanced is 9. With 16 possible input combinations, the function can return all 0s for the first 8 ouputs and on the ninth, return 1. This indicates it is balanced. Thus, the max number of times you need to call f to be 100% certain it is balanced is `9`.




### What is the maximum number of times you need to call f to be 100% certain whether it is constant?

`Constant Functions` : A function always either return 1 or always either returns 0.

**Minimum and Maximum**

With a function that accepts 4 argumnets as input, both the minimum and maximum number of calls it would take to determine if f is a constant function is also `9`. With 16 different input combinations if a function is called 8 times and all the inputs yield 1, without the 9th combination it cannot yet be determined if the function is constant ro balanced. On the 9th function call, if it were to return 1 again, it is constant as more than half of the inputs were 1.


## Problem 3: Quantum Oracles


## Problem 4: Deutsch's Algorithm with Qiskit


## Problem 5: Scaling to the Deutsch–Jozsa Algorithm
